# Дневные когорты

In [ ]:
import pyodbc
import pandas as pd
import numpy as np
from google.cloud import bigquery
import dill
from tqdm import tqdm_notebook
from datetime import datetime
from datetime import date
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')


client = bigquery.Client.from_service_account_json(r'xxx.json')
print('Укажите день базовой когорты в формате "20190901"')
start_period = str(input())
all_per = []

q_statga_first = ("SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX = '{}' AND  hits.page.pagePath LIKE '%opentrainer%'".format(start_period))
qs_j = client.query(q_statga_first)
first_cohort_clids = qs_j.to_dataframe()
first_cohort_clids_list0 = [ "'"+str(e)+"'" for e in list(first_cohort_clids['clientid'])]
first_cohort_clids_list = ', '.join(first_cohort_clids_list0)

len_cohs = []
len_cohs.append(first_cohort_clids.shape[0])

len_period = int(input())
for e in range(1,len_period+1):
    all_per.append((datetime.strptime(start_period, '%Y%m%d') + timedelta(e)).strftime('%Y%m%d'))
    


In [ ]:
for m in tqdm_notebook(all_per):
    QUERY1 = ("SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX = '{}' AND  hits.page.pagePath LIKE '%opentrainer%' AND clientid IN ({})".format(m,first_cohort_clids_list))
    query_job1 = client.query(QUERY1)
    df_all = query_job1.to_dataframe()
    len_cohs.append(df_all.shape[0])

df_len = pd.DataFrame({'Длина когорты':len_cohs}).reset_index()
df_len.rename(columns={'index': 'День'}, inplace=True)
df_len['%'] = df_len['Длина когорты']*100/first_cohort_clids.shape[0]
df_len

In [ ]:
df_len.head(10)

# Недельные и месячные когорты

In [ ]:
import pyodbc
import pandas as pd
import numpy as np
from google.cloud import bigquery
import dill
from tqdm import tqdm_notebook
from datetime import datetime
from datetime import date
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')


client = bigquery.Client.from_service_account_json(r'api-open-broker-85cd95f295fc.json')
print('Укажите день начала и конца базовой когорты в формате "20190901"')
start_period = str(input())
end_period = str(input())
all_per = []

q_statga_first = ("SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND '{}' AND  hits.page.pagePath LIKE '%opentrainer%'".format(start_period,end_period))
qs_j = client.query(q_statga_first)
first_cohort_clids = qs_j.to_dataframe()
first_cohort_clids_list0 = [ "'"+str(e)+"'" for e in list(first_cohort_clids['clientid'])]
first_cohort_clids_list = ', '.join(first_cohort_clids_list0)
len_cohs = []
len_cohs.append(first_cohort_clids.shape[0])

print('Укажите какие когорты брать - недельные или месячные. Напишите: month или week')
mode=str(input())
print('Укажите длительность горизонта')
len_period = int(input())
start_per = []
end_per = []
for e in range(1,len_period+1):
    if mode == 'week':
        start_per.append((datetime.strptime(start_period, '%Y%m%d') + timedelta(e*7)).strftime('%Y%m%d'))
        end_per.append((datetime.strptime(end_period, '%Y%m%d') + timedelta(e*7)).strftime('%Y%m%d'))
    if mode == 'month':
        start_per.append((datetime.strptime(start_period, '%Y%m%d') + timedelta(e*30)).strftime('%Y%m%d'))
        end_per.append((datetime.strptime(end_period, '%Y%m%d') + timedelta(e*30)).strftime('%Y%m%d'))

In [ ]:
for index,m in tqdm_notebook(enumerate(start_per)):
    QUERY1 = ("SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND '{}' AND  hits.page.pagePath LIKE '%opentrainer%' AND clientid IN ({})".format(m,end_per[index],first_cohort_clids_list))
    query_job1 = client.query(QUERY1)
    df_all = query_job1.to_dataframe()
    len_cohs.append(df_all.shape[0])

df_len = pd.DataFrame({'Длина когорты':len_cohs}).reset_index()
df_len.rename(columns={'index': mode}, inplace=True)
df_len['%'] = df_len['Длина когорты']*100/first_cohort_clids.shape[0]
df_len.head(10)